In [ ]:
getwd()

In [ ]:
#READ CSV FILE AND CONVERT TO ONE DATA FRAME
col_name <- c("","Trip.Start.Timestamp","Trip.Seconds","Trip.Miles","Pickup.Community.Area","Dropoff.Community.Area","Company")
setwd("/Users/rpgen/Documents/CS424/project3/bigYellowTaxi/Data")
myfiles <- list.files(pattern="*.csv", full.names=TRUE)
myfiles
data <- do.call(rbind, lapply(myfiles, read.csv, header = FALSE))
colnames(data) <- col_name



In [ ]:
data

In [ ]:
library(lubridate)

n_date <- parse_date_time(data$Trip.Start.Timestamp,
                orders = 'mdY IMS %p', truncated = 3) #PARSE DATE FROM TIME STAMP
data$new_date <- n_date
data$Hour <- hour(data$new_date)
data

In [ ]:
community_list <- c("Rogers Park", "West Ridge", "Uptown", 
          "Lincoln Square", "North Center", "Lake View",
         "Lincoln Park", "Near North Side", "Edison Park",
         "Norwood Park", "Jefferson Park", "Forest Glen",
         "North Park", "Albany Park", "Portage Park",
         "Irving Park", "Dunning", "Montclare", "Belmont Cragin",
         "Hermosa", "Avondale", "Logan Square", "Humboldt Park",
         "West Town", "Austin", "West Garfield Park",
         "East Garfield Park", "Near West Side", "North Lawndale",
         "South Lawndale", "Lower West Side", "Loop", "Near South Side",
         "Armour Square", "Douglas", "Oakland", "Fuller Park", 
         "Grand Boulevard", "Kenwood", "Washington Park", "Hyde Park",
         "Woodlawn", "South Shore", "Chatham", "Avalon Park", "South Chicago",
         "Burnside", "Calumet Heights", "Roseland", "Pullman", "South Deering",
         "East Side", "West Pullman", "Riverdale", "Hegewisch", "Garfield Ridge",
         "Archer Heights", "Brighton Park", "Mckinley Park", "Bridgeport",
         "New City", "West Elsdon", "Gage Park", "Clearing", "West Lawn",
         "Chicago Lawn", "West Englewood", "Englewood", "Greater Grand Crossing",
         "Ashburn", "Auburn Gresham", "Beverly", "Washington Heights",
         "Mount Greenwood", "Morgan Park", "Ohare", "Edgewater") # R IS INDEX STARTING 1 NOT 0

company_list <- c("1085 - 72312 N and W Cab Co", "1469 - 64126 Omar Jada","2092 - 61288 Sbeih company", "24 Seven Taxi",
                  "2733 - 74600 Benny Jona", "3011 - 66308 JBL Cab Inc.", "3094 - 24059 G.L.B. Cab Co", 
                  "312 Medallion Management Corp", "3556 - 36214 RC Andrews Cab", "3591 - 63480 Chuks Cab", 
                  "3620 - 52292 David K. Cab Corp.", "3623 - 72222 Arrington Enterprises", 
                  "3721 - Santamaria Express, Alvaro Santamaria", "4053 - 40193 Adwar H. Nikola", 
                  "4623 - 27290 Jay Kim", "5 Star Taxi", "5006 - 39261 Salifu Bawa", "5062 - 34841 Sam Mestas",
                  "5074 - 54002 Ahzmi Inc", "5874 - 73628 Sergey Cab Corp.", "6574 - Babylon Express Inc.",
                  "6742 - 83735 Tasha ride inc", "6743 - 78771 Luhak Corp", "American United", 
                  "American United Taxi Affiliation", "Blue Diamond", "Blue Ribbon Taxi Association Inc.",
                  "Checker Taxi", "Checker Taxi Affiliation", "Chicago Carriage Cab Corp", 
                  "Chicago Independents", "Chicago Medallion Management", "Chicago Star Taxicab", 
                  "Chicago Taxicab", "Choice Taxi Association", "City Service", "Flash Cab", "Globe Taxi", 
                  "Gold Coast Taxi", "KOAM Taxi Association", "Leonard Cab Co", "Medallion Leasin", 
                  "Metro Jet Taxi A", "Nova Taxi Affiliation Llc", "Patriot Taxi Dba Peace Taxi Associat", 
                  "Setare Inc", "Star North Management LLC", "Sun Taxi", "Taxi Affiliation Service Yellow", 
                  "Taxi Affiliation Services", "Taxicab Insurance Agency, LLC", "Top Cab Affiliation", "Yellow Cab",
                  "Petani Cab Corp", "U Taxicab")

#All (City of Chicago)
community <- "All (City of Chicago)"
trip <- "To"
#All Taxis
#Metro Jet Taxi A
company <- "All Taxis"
units <- "MI"

In [ ]:
print(community_list[59]) #GRABBING COMMUNITY BY REGION ID
match("Dunning", community_list) #GETTING REGION ID BY COMMUNITY NAME


In [ ]:
upper_com <- NULL
for(i in community_list){
    upper_com <- append(upper_com, toupper(i))
}

df<-data.frame(upper_com)
df$pop <- 25
for(i in 1:length(df[,1])){
    print(i)
    df[i,2] <- df[i,2] + i
}

df

In [ ]:
sort(unique(data$Company))

In [ ]:
updateData <- data.frame(data)

if(community != "All (City of Chicago)"){
    if(trip == "To"){
        updateData <- subset(updateData, updateData$Dropoff.Community.Area == match(community, community_list))
    }
    else{ #FROM
        updateData <- subset(updateData, updateData$Pickup.Community.Area == match(community, community_list))
    }
}
if(company != "All Taxis"){
    updateData <- subset(updateData, updateData$Company == company)
}


In [ ]:
updateData

In [ ]:
#Aggregate by day
noFilter_byDay <- aggregate(updateData[,1], by=list(date(updateData$new_date)), FUN=length)
colnames(noFilter_byDay) <- c("Date", "Rides")
noFilter_byDay

In [ ]:
#Aggregate by hour
noFilter_byHour <- aggregate(updateData[,1], by=list(updateData$Hour), FUN=length)
colnames(noFilter_byHour) <- c("Hour", "Rides")
noFilter_byHour

In [ ]:
#Aggregate by weekday
noFilter_byDayOfWeek <- aggregate(updateData[,1], by=list(wday(updateData$new_date, week_start=1)), FUN=length)
colnames(noFilter_byDayOfWeek) <- c("Day Of Week", "Rides")
#** STARTS MONDAY TO SUNDAY WITH 1 BEING MONDAY **
noFilter_byDayOfWeek

In [ ]:
#Aggregate by binned mileage 

#Add to global
mile_bin <- c("0.5 to < 10", "10 to < 20", "20 to < 30", "30 to < 40", "40 to < 50", "50 to < 60", "60 to < 70", "70 to < 80", "80 to < 90", "90 to 100")

#Actual code to get data frame
bin_data = data.frame(updateData)
bin_data$Mile_Bin <- NA
bin_data$Mile_Bin[bin_data$Trip.Miles >= 0.5 & bin_data$Trip.Miles < 10] <- mile_bin[1]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 10 & bin_data$Trip.Miles < 20] <- mile_bin[2]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 20 & bin_data$Trip.Miles < 30] <- mile_bin[3]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 30 & bin_data$Trip.Miles < 40] <- mile_bin[4]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 40 & bin_data$Trip.Miles < 50] <- mile_bin[5]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 50 & bin_data$Trip.Miles < 60] <- mile_bin[6]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 60 & bin_data$Trip.Miles < 70] <- mile_bin[7]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 70 & bin_data$Trip.Miles < 80] <- mile_bin[8]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 80 & bin_data$Trip.Miles < 90] <- mile_bin[9]
bin_data$Mile_Bin[bin_data$Trip.Miles >= 90 & bin_data$Trip.Miles <= 100] <- mile_bin[10]
bin_data$Mile_Bin <- as.factor(bin_data$Mile_Bin)

noFilter_byBinnedMileage <- aggregate(bin_data[,1], by=list(bin_data$Mile_Bin), FUN=length)
colnames(noFilter_byBinnedMileage) <- c("Miles", "Rides")
noFilter_byBinnedMileage

In [ ]:
#Aggregate by Trip Time
print(min(updateData$Trip.Seconds))
max(updateData$Trip.Seconds)

# #Add to global
time_bin <- c("1 minute to < 5 minutes", "5 minute to < 10 minutes", "10 minute to < 30 minutes", "30 minute to < 1 hr", "1 hr to < 2 hr", "2 hr to < 3 hr", "3 hr to < 4 hr", "4 hr to 5 hr")

# #Actual code to get data frame
bin_data = data.frame(updateData)
bin_data$Time_Bin <- NA
bin_data$Time_Bin[bin_data$Trip.Seconds >= 60 & bin_data$Trip.Seconds < 300] <- time_bin[1]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 300 & bin_data$Trip.Seconds < 600] <- time_bin[2]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 600 & bin_data$Trip.Seconds < 1800] <- time_bin[3]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 1800 & bin_data$Trip.Seconds < 3600] <- time_bin[4]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 3600 & bin_data$Trip.Seconds < 7200] <- time_bin[5]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 7200 & bin_data$Trip.Seconds < 10800] <- time_bin[6]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 10800 & bin_data$Trip.Seconds < 14400] <- time_bin[7]
bin_data$Time_Bin[bin_data$Trip.Seconds >= 14400 & bin_data$Trip.Seconds < 18000] <- time_bin[8]
bin_data$Time_Bin <- factor(bin_data$Time_Bin, levels = time_bin, ordered = TRUE) 
noFilter_byBinnedTime <- aggregate(bin_data[,1], by=list(bin_data$Time_Bin), FUN=length)
colnames(noFilter_byBinnedTime) <- c("Time", "Rides")
noFilter_byBinnedTime



In [ ]:
summary(noFilter_byBinnedTime)

In [ ]:
# Dataframe for the percentage of rides going to/from each community
# NEEDS TO ACCOUNT THE TO/FROM FILTER // done
# INTEGRATE WITH TAXI // done
# KEEP IN MIND TO KEEP THE ORDER OF COMMUNITY ARRAY // done
eachCommunity <- data.frame(data)
comm_range <- 1:77

if(company != "All Taxis"){
    eachCommunity <- subset(eachCommunity, eachCommunity$Company == company)
}

if(trip == "To"){
    eachCommunity <- aggregate(eachCommunity[,1], by=list(eachCommunity$Dropoff.Community.Area), FUN=length)
}else{
    eachCommunity <- aggregate(eachCommunity[,1], by=list(eachCommunity$Pickup.Community.Area), FUN=length)
}

colnames(eachCommunity) <- c("Community", "Percentage")
#Fill in missing communities with 0's
for(i in 1:77){
    hasCommunity <- FALSE
    for(j in 1:length(eachCommunity[,1])){
        if(eachCommunity[j,1] == i){
            hasCommunity <- TRUE
            break
        }
    }
        
    if(!hasCommunity){
        eachCommunity[nrow(eachCommunity) + 1,] = c(i, 0)
    }
}

eachCommunity <- eachCommunity[order(eachCommunity$Community),]
rownames(eachCommunity) <- 1:nrow(eachCommunity)

#CONVERT COMMUNITY ID TO NAMES
for(i in 1:77){
    eachCommunity[i,1] <- community_list[i]
}

#Table them as percentages
count_sum <- sum(eachCommunity$Percentage)
eachCommunity$Percentage <- eachCommunity$Percentage/count_sum
eachCommunity
#NOTE BAR CHART NEEDS TO SHOW COMMUNITIES ALPHABETICALLY
